# DreamBooth Training on Selfies
This notebook trains Stable Diffusion using the DreamBooth method on a dataset of 41 selfies to generate professional-looking images.

In [ ]:
# Install necessary libraries
!pip install torch diffusers datasets pillow matplotlib huggingface_hub

In [ ]:
import os
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from datasets import load_dataset
from PIL import Image
import matplotlib.pyplot as plt
from huggingface_hub import HfApi, HfFolder, Repository

## Load Dataset

In [ ]:
# Load the dataset
dataset = load_dataset('selfies')
images = [Image.open(f'sample_image_{i:02d}.png') for i in range(1, 42)]

## Initialize Stable Diffusion Pipeline

In [ ]:
# Initialize the pipeline
model_id = 'CompVis/stable-diffusion-v1-4'
pipe = StableDiffusionPipeline.from_pretrained(model_id)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to('cuda')

## DreamBooth Training Loop

In [ ]:
# Training loop using DreamBooth
for i, img in enumerate(images):
    img = img.resize((512, 512))
    img = torch.tensor(img).unsqueeze(0).to('cuda')
    pipe.train(img)
    if i % 10 == 0:
        print(f'Training step {i} completed')

## Generate Images

In [ ]:
# Generate images
num_samples = 30
generated_images = []
positive_prompt = 'Professional headshot of a 42-year-old man with a beard and glasses, looking confident and approachable, studio lighting, high resolution, sharp focus'
negative_prompt = 'blurry, distorted, unrealistic, low quality, artifacts'
for _ in range(num_samples):
    image = pipe(positive_prompt, negative_prompt=negative_prompt).images[0]
    generated_images.append(image)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

## Save and Upload Model to Hugging Face

In [ ]:
# Save the model
model_path = 'dreambooth_model'
pipe.save_pretrained(model_path)

# Upload to Hugging Face
api = HfApi()
token = HfFolder.get_token()
repo_name = 'your-username/dreambooth-model'
repo = Repository(local_dir=model_path, clone_from=repo_name, use_auth_token=token)
repo.push_to_hub()